# README - Phân Tích Dữ Liệu

## Mô Tả Dự Án

Dự án này cung cấp các công cụ mạnh mẽ để phân tích dữ liệu, bao gồm các chức năng tổng hợp thông tin, trực quan hóa dữ liệu và phân tích thống kê trên các cột trong DataFrame. Các chức năng chính của dự án giúp bạn hiểu rõ hơn về dữ liệu của mình và thực hiện phân tích dữ liệu một cách hiệu quả.

### Các Chức Năng Chính:
1. **Tổng hợp các đặc trưng thống kê** cho các cột dạng số và phân loại.
2. **Tạo bảng tổng quan** về dữ liệu, bao gồm các thông tin về các giá trị trùng lặp, thiếu, và các đặc trưng khác.
3. **Vẽ biểu đồ đơn biến (univariate)** cho các cột dữ liệu.
4. **Vẽ biểu đồ phân tích đa biến (multivariate)** giữa một cột chính và các cột còn lại.

---

In [1]:
#| default_exp aulys

In [2]:
#| hide
import os
import sys

In [3]:
#| hide
ROOT_PATH = ""
for path in os.getcwd().split("\\")[:-1]:
    ROOT_PATH += f"{path}/"
sys.path.insert(1, ROOT_PATH)

In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
import os
import sys

import yaml
import pandas as pd
import lets_plot as lp
from IPython.display import Markdown

import config
import src.utils as utils
import src.chart as chart
import src.data as data

In [6]:
#| hide
with open(os.path.join(ROOT_PATH, f'data/interim/mssql.yaml'), 'r') as f:
    mssql_config = yaml.safe_load(f)

with open(os.path.join(ROOT_PATH, f'data/interim/schema.yaml'), 'r') as f:
    schema = yaml.safe_load(f)

with open(os.path.join(ROOT_PATH, f'data/interim/map_cols.yaml'), 'r', encoding="utf8") as f:
    map_cols = yaml.safe_load(f)

table_name = 'ODS_DW_CARD_TRANSACTION_FCT'

df = pd.read_csv(
    os.path.join(ROOT_PATH, f'data/raw/{config.TODAY}/{table_name}.csv')
)

ass_cfg = {
    'process_dt': lambda x: pd.to_datetime(x['process_dt']),
    'process_month': lambda x: x['process_dt'].dt.to_period(freq='M')
}
df_agg = (
    df[
        ['account_number', 'process_dt'] 
        + schema[table_name]['cat_cols']
        + schema[table_name]['num_cols']
    ]
    .fillna('<unknow>')
    .assign(
        **ass_cfg
    )
)

df_agg = df_agg[
    df_agg['process_dt'] > (pd.Timestamp(config.TODAY).to_period('M') - 3).end_time
]

df_agg[schema[table_name]['cat_cols']] = df_agg[schema[table_name]['cat_cols']].astype(str)

with open(
    os.path.join(ROOT_PATH, f'data/interim/bin_cols.yaml'),
    'r'
) as f:
        bin_cols = yaml.safe_load(f)

In [7]:
#| export
def stast_fea(df, idx_col, cat_cols, num_cols):
    """
    Tổng hợp các đặc trưng dạng số bằng các phép thống kê:
    - Đối với dữ liệu numerical:
        * Trung bình (`mean`)
        * Giá trị lớn nhất (`max`)
        * Giá trị nhỏ nhất (`min`)
        * Tổng (`sum`)
        * Trung vị (`median`)
        * Hiệu số giữa max và min (`diff`)
    - Đối với categorical:
        * Số lượng giá trị khác nhau (`nunique`)
        * Giá trị xuất hiện nhiều nhất (`mode`)
        * Tần suất xuất hiện của từng giá trị trong cột phân loại
    
    Args:
        df (pd.DataFrame): Dữ liệu đầu vào
        idx_col (str): Cột index (ID nhóm)
        num_cols (list): Danh sách các cột số để tổng hợp
        cat_cols (list): Danh sách các cột phân loại để tổng hợp

    Returns:
        pd.DataFrame: DataFrame chứa các feature tổng hợp
    """
    return data.agg_fea(
        df=df, idx_col=idx_col, cat_cols=cat_cols, num_cols=num_cols
    )

## Tổng Hợp Đặc Trưng Thống Kê

Hàm `stast_fea(df, idx_col, cat_cols, num_cols)` thực hiện việc tính toán các thống kê cho các cột số và phân loại trong DataFrame. Các thống kê này giúp bạn hiểu rõ hơn về dữ liệu của mình.

### Thống Kê Cho Cột Số:
- **Trung bình (mean)**: Tính giá trị trung bình của các phần tử trong cột.
- **Giá trị lớn nhất (max)**: Tìm giá trị lớn nhất trong cột.
- **Giá trị nhỏ nhất (min)**: Tìm giá trị nhỏ nhất trong cột.
- **Tổng (sum)**: Tính tổng các giá trị trong cột.
- **Trung vị (median)**: Tính giá trị trung vị của các phần tử trong cột.
- **Hiệu số giữa giá trị lớn nhất và nhỏ nhất (diff)**: Tính hiệu số giữa giá trị lớn nhất và giá trị nhỏ nhất trong cột.

### Thống Kê Cho Cột Phân Loại:
- **Số lượng giá trị khác nhau (nunique)**: Đếm số lượng các giá trị duy nhất trong cột.
- **Giá trị xuất hiện nhiều nhất (mode)**: Tìm giá trị xuất hiện nhiều nhất trong cột.
- **Tần suất xuất hiện của từng giá trị trong cột phân loại**: Hiển thị số lần xuất hiện của từng giá trị trong cột phân loại.

### Tham Số Đầu Vào:
- **df**: DataFrame chứa dữ liệu cần tính toán thống kê.
- **idx_col**: Cột chỉ mục (ID nhóm) để phân nhóm các giá trị.
- **cat_cols**: Danh sách các cột phân loại (categorical) cần tính toán thống kê.
- **num_cols**: Danh sách các cột số (numerical) cần tính toán thống kê.

### Ví Dụ Sử Dụng

Giả sử bạn có một DataFrame `df` và muốn tính toán các thống kê cho các cột phân loại và số, với cột `account_number` là chỉ mục:

```python
stast_fea(df, idx_col='account_number', cat_cols=['category1', 'category2'], num_cols=['amount', 'quantity'])


Ví dụ sử dụng

In [8]:
# df_acc = stast_fea(
#     df=df_agg, 
#     idx_col='account_number', 
#     cat_cols=schema[table_name]['cat_cols'], 
#     num_cols=schema[table_name]['num_cols']
# )

In [9]:
#| export
def overview_table(df):
    """
    Tạo bảng tổng quan về dữ liệu, bao gồm:
    - Số lượng giá trị trùng lặp (`Count_duplicate`)
    - Số lượng giá trị bị thiếu (`Count_missing`)
    - Tỷ lệ phần trăm giá trị bị thiếu (`Percent_missing`)
    - Số lượng giá trị duy nhất (`Count_distinct`)
    - Số lượng giá trị bằng 0 (`Count_zero`, chỉ áp dụng cho kiểu số)
    - Tỷ lệ phần trăm giá trị bằng 0 (`Percent_zero`, chỉ áp dụng cho kiểu số)
    
    Args:
        df (pd.DataFrame, optional): DataFrame đầu vào.
    
    Returns:
        pd.DataFrame: DataFrame chứa thông tin tổng quan về các cột.
    """
    return utils.overview_table(df=df)

## Tạo Bảng Tổng Quan Về Dữ Liệu

Hàm `overview_table(df)` giúp bạn tạo ra một bảng tổng quan cho các cột trong DataFrame, cung cấp các thông tin cơ bản về dữ liệu của bạn.

### Các Thông Tin Tổng Quan:
Bảng tổng quan sẽ bao gồm các thông tin sau cho mỗi cột trong DataFrame:
- **Số lượng giá trị trùng lặp (Count_duplicate)**: Số lượng các giá trị bị trùng lặp trong cột.
- **Số lượng giá trị bị thiếu (Count_missing)**: Số lượng các giá trị bị thiếu (NaN) trong cột.
- **Tỷ lệ phần trăm giá trị bị thiếu (Percent_missing)**: Tỷ lệ phần trăm các giá trị bị thiếu trong cột.
- **Số lượng giá trị duy nhất (Count_distinct)**: Số lượng các giá trị khác nhau trong cột.
- **Số lượng giá trị bằng 0 (Count_zero)**: Số lượng giá trị bằng 0 trong cột (chỉ áp dụng cho các cột có kiểu số).
- **Tỷ lệ phần trăm giá trị bằng 0 (Percent_zero)**: Tỷ lệ phần trăm giá trị bằng 0 trong cột (chỉ áp dụng cho các cột có kiểu số).

### Tham Số Đầu Vào:
- **df**: DataFrame chứa dữ liệu cần tính toán thống kê.

### Ví Dụ Sử Dụng

Giả sử bạn có một DataFrame `df` và muốn tạo bảng tổng quan về dữ liệu:

In [10]:
# overview_table(df=df_acc)

In [11]:
#| export
def plot_univariate(df, bin_cols):
    """
    Vẽ biểu đồ đơn biến (univariate) cho tất cả các cột trong DataFrame.

    - Nếu là cột phân loại (chuỗi): vẽ biểu đồ tần suất.
    - Nếu là cột số: chia bin rồi vẽ biểu đồ.

    Args:
        df (pd.DataFrame): Dữ liệu đầu vào (cột đầu tiên thường là ID, sẽ bỏ qua).
        bin_cols (dict): Từ điển chứa các giá trị phân bin cho các cột số.
    """
    display(chart.plot_univariate(df=df, bin_cols=bin_cols))

## Vẽ Biểu Đồ Đơn Biến

Hàm `plot_univariate(df)` vẽ biểu đồ đơn biến cho tất cả các cột trong DataFrame:

- **Các cột phân loại (chuỗi)** sẽ được vẽ biểu đồ tần suất.
- **Các cột số** sẽ được chia thành các bin và vẽ biểu đồ phân phối.

### Tham Số Đầu Vào:
- **df**: DataFrame chứa dữ liệu cần tính toán thống kê.
- **bin_cols**: Từ điển chứa các giá trị phân bin cho các cột số.

### Cấu Hình Bin

Để cấu hình các mốc bin cho các cột số, bạn cần tạo một file `config.yaml` trong thư mục `data/interim/` với cấu trúc sau:

```json
{
    "tên_cột_1": [x1, x2, x3, ...],
    "tên_cột_2": [x1, x2, x3, ...]
}


# Trong đó: x1, x2, x3 là các mốc bin muốn chia

#Ví dụ
{
    "age": [20, 30, 40, 50, 60],
    "income": [1000, 2000, 3000, 4000, 5000]
}


Ví dụ sử dụng

In [12]:
# plot_univariate(df=df_acc, bin_cols=bin_cols)

In [13]:
#| export
def plot_multivariate(df, column, bin_cols):
    """
    Vẽ biểu đồ phân tích đa biến giữa một cột chính (`column`) với tất cả các cột còn lại.
    Tùy vào kiểu dữ liệu (chuỗi hoặc số), cột được chia bin hoặc chuyển thành chuỗi để trực quan hóa.

    Args:
        df (pd.DataFrame): Dữ liệu đầu vào.
        column (str): Cột chính cần phân tích.
        bin_cols (dict): Dictionary chứa thông tin bin cho các cột số.

    Returns:
        None: Hiển thị biểu đồ crosstab cho từng cặp biến.
    """
    display(chart.plot_multivariate(df=df, column=column, bin_cols=bin_cols))

## Vẽ Biểu Đồ Phân Tích Đa Biến

Hàm `plot_multivariate(df, column)` vẽ biểu đồ phân tích đa biến giữa một cột chính (`column`) và tất cả các cột còn lại trong DataFrame.

### Mô Tả
- Hàm sẽ phân tích mối quan hệ giữa **cột chính** (`column`) và các cột còn lại trong DataFrame.
- Tùy thuộc vào kiểu dữ liệu của các cột còn lại (cột số hoặc cột phân loại), các cột này sẽ được xử lý và chuyển đổi phù hợp để hiển thị biểu đồ **crosstab** (ma trận chéo).

### Cách Hoạt Động:
- Nếu các cột còn lại là **cột phân loại (chuỗi)**, hàm sẽ chuyển đổi các giá trị trong cột thành các nhóm và vẽ biểu đồ crosstab.
- Nếu các cột còn lại là **cột số** (numerical), hàm sẽ phân chia các giá trị thành các bin hoặc hiển thị theo các nhóm để thể hiện mối quan hệ với cột chính.

### Tham Số Đầu Vào:
- **df**: DataFrame chứa dữ liệu cần tính toán thống kê.
- **column**: Cột chính cần phân tích.
- **bin_cols**: DaDictionary chứa thông tin bin cho các cột số.

### Ví Dụ Sử Dụng

Giả sử bạn muốn phân tích mối quan hệ giữa cột `mean_oper_request_amount` (một cột số) và các cột còn lại trong DataFrame, bạn có thể sử dụng hàm như sau:

Chú ý: nếu muốn chỉnh bins thì có thể chia cột config tương tự

In [14]:
# plot_multivariate(df=df_acc, column='mean_oper_request_amount')

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()